# feats using polar yay

In [2]:
import logging
import sys
import time
from pathlib import Path

# import pandas as pd
import polars as pl
import pybaseball as pb
import yaml

logger = logging.getLogger("featurize")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(
    logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
)
logger.addHandler(handler)
cfg = pl.Config()
cfg.set_tbl_rows(2000)

polars.config.Config

In [3]:
with open("../params.yaml", "r") as file:
    params = yaml.safe_load(file)
input_file_path = params["featurize"]["input_data_path"]

df = pl.read_parquet("../" / Path(input_file_path))

In [4]:
df = df.with_columns(
    df.select(
        pl.col(pl.String).exclude(["player_name"]).cast(pl.Categorical).to_physical()
    ),
)

In [21]:
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=False,
)
# create target variable
df = df.with_columns(
    df.select(pl.col("pitch_type").shift().alias("next_pitch")),
).drop_nulls("next_pitch")

In [22]:
df.select(pl.col(["pitch_type", "next_pitch"])).head(20)

pitch_type,next_pitch
i64,i64
5,1
1,5
3,1
5,3
4,5
1,4
1,1
1,1
1,1


In [7]:
df = df.with_columns(
    (pl.col("balls").cast(pl.String) + " - " + pl.col("strikes").cast(pl.String))
    .alias("count")
    .cast(pl.Categorical)
    .to_physical()
)
df = df.drop(["balls", "strikes"])

In [8]:
print(df.select("count").describe())
df.head()


shape: (9, 2)
┌────────────┬──────────┐
│ statistic  ┆ count    │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ count      ┆ 39819.0  │
│ null_count ┆ 0.0      │
│ mean       ┆ 3.484091 │
│ std        ┆ 3.230247 │
│ min        ┆ 0.0      │
│ 25%        ┆ 0.0      │
│ 50%        ┆ 3.0      │
│ 75%        ┆ 6.0      │
│ max        ┆ 11.0     │
└────────────┴──────────┘


pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,hit_location,bb_type,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,ax,ay,az,hit_distance_sc,launch_speed,launch_angle,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,next_pitch,count
u32,datetime[μs],f64,f64,f64,str,i64,i64,u32,u32,f64,u32,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,u32,u32
1,2021-03-16 00:00:00,93.0,-1.94,5.32,"""Nola, Aaron""",543807,605400,1,1,null,null,null,null,null,0,1,null,null,-12.348547,30.73969,-16.651987,170.0,91.9,65.0,2227.0,6.9,641572,53.58,null,null,null,null,null,null,null,1,1,0,0,0,0,0,0,231.0,0.0,null,null,null,5,0
5,2021-03-16 00:00:00,83.5,-2.0,5.26,"""Nola, Aaron""",543807,605400,1,1,null,null,null,null,null,0,1,null,null,-14.455206,25.331797,-31.088718,null,null,null,1398.0,6.5,641572,53.98,null,null,null,null,null,null,null,1,2,0,0,0,0,0,0,256.0,0.0,null,null,null,1,1
1,2021-03-16 00:00:00,93.9,-1.89,5.3,"""Nola, Aaron""",543807,605400,1,2,null,null,null,null,null,0,1,null,null,-13.677609,29.965809,-15.211607,null,null,null,2175.0,7.0,641572,53.52,null,null,null,null,null,null,null,1,3,0,0,0,0,0,0,228.0,0.0,null,null,null,3,2
3,2021-03-16 00:00:00,79.6,-1.92,5.42,"""Nola, Aaron""",543807,605400,1,2,null,null,null,null,null,0,1,null,null,9.351337,24.548919,-39.815268,null,null,null,2603.0,6.6,641572,53.89,null,null,null,null,null,null,null,1,4,0,0,0,0,0,0,52.0,0.0,null,null,null,5,3
5,2021-03-16 00:00:00,83.8,-2.08,5.26,"""Nola, Aaron""",543807,605400,1,1,null,null,null,null,null,0,1,null,null,-13.566679,23.380459,-33.125139,7.0,93.4,-18.0,1382.0,6.6,641572,53.9,null,null,null,null,null,null,null,1,5,0,0,0,0,0,0,255.0,0.0,null,null,null,4,4


In [9]:
df = df.with_columns(df.select(["on_1b", "on_2b", "on_3b"]).fill_null(-1))
df = df.with_columns(
    pl.col("on_1b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_1b"),
    pl.col("on_2b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_2b"),
    pl.col("on_3b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_3b"),
)
df = df.with_columns(
    (
        pl.col("on_1b").cast(pl.String)
        + "/"
        + pl.col("on_2b").cast(pl.String)
        + "/"
        + pl.col("on_3b").cast(pl.String)
    )
    .alias("base_state")
    .cast(pl.Categorical)
    .to_physical()
)
df = df.drop(["on_1b", "on_2b", "on_3b"])

In [10]:
print(df.select("base_state").describe())

df.head()

shape: (9, 2)
┌────────────┬────────────┐
│ statistic  ┆ base_state │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ count      ┆ 39819.0    │
│ null_count ┆ 0.0        │
│ mean       ┆ 0.94593    │
│ std        ┆ 1.5139     │
│ min        ┆ 0.0        │
│ 25%        ┆ 0.0        │
│ 50%        ┆ 0.0        │
│ 75%        ┆ 2.0        │
│ max        ┆ 7.0        │
└────────────┴────────────┘


pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,hit_location,bb_type,outs_when_up,inning,hc_x,hc_y,ax,ay,az,hit_distance_sc,launch_speed,launch_angle,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,next_pitch,count,base_state
u32,datetime[μs],f64,f64,f64,str,i64,i64,u32,u32,f64,u32,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,u32,u32,u32
1,2021-03-16 00:00:00,93.0,-1.94,5.32,"""Nola, Aaron""",543807,605400,1,1,null,null,0,1,null,null,-12.348547,30.73969,-16.651987,170.0,91.9,65.0,2227.0,6.9,641572,53.58,null,null,null,null,null,null,null,1,1,0,0,0,0,0,0,231.0,0.0,null,null,null,5,0,0
5,2021-03-16 00:00:00,83.5,-2.0,5.26,"""Nola, Aaron""",543807,605400,1,1,null,null,0,1,null,null,-14.455206,25.331797,-31.088718,null,null,null,1398.0,6.5,641572,53.98,null,null,null,null,null,null,null,1,2,0,0,0,0,0,0,256.0,0.0,null,null,null,1,1,0
1,2021-03-16 00:00:00,93.9,-1.89,5.3,"""Nola, Aaron""",543807,605400,1,2,null,null,0,1,null,null,-13.677609,29.965809,-15.211607,null,null,null,2175.0,7.0,641572,53.52,null,null,null,null,null,null,null,1,3,0,0,0,0,0,0,228.0,0.0,null,null,null,3,2,0
3,2021-03-16 00:00:00,79.6,-1.92,5.42,"""Nola, Aaron""",543807,605400,1,2,null,null,0,1,null,null,9.351337,24.548919,-39.815268,null,null,null,2603.0,6.6,641572,53.89,null,null,null,null,null,null,null,1,4,0,0,0,0,0,0,52.0,0.0,null,null,null,5,3,0
5,2021-03-16 00:00:00,83.8,-2.08,5.26,"""Nola, Aaron""",543807,605400,1,1,null,null,0,1,null,null,-13.566679,23.380459,-33.125139,7.0,93.4,-18.0,1382.0,6.6,641572,53.9,null,null,null,null,null,null,null,1,5,0,0,0,0,0,0,255.0,0.0,null,null,null,4,4,0


In [11]:
print(df.shape)

# # find columns with null percentage > 10%
# # Calculate null percentages for all columns
# null_percentages = df.select(pl.all().is_null().sum() / df.height)

# # Filter columns where null percentage > 10%
# columns_with_too_many_nulls = (
#     null_percentages.melt().filter(pl.col("value") > 0.1).select("variable")
# )

# print("Columns with >10% null values:")
# print(columns_with_too_many_nulls)

# drop columns with too many nulls
df = df.drop(
    df.select(pl.all().is_null().sum() / df.height)
    .unpivot()
    .filter(pl.col("value") > 0.05)
    .select("variable")
    .to_series()
    .to_list()
)
print(df.shape)
df.head()

(39819, 49)
(39819, 33)


pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,outs_when_up,inning,ax,ay,az,release_spin_rate,release_extension,game_pk,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,next_pitch,count,base_state
u32,datetime[μs],f64,f64,f64,str,i64,i64,u32,u32,i64,i64,f64,f64,f64,f64,f64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,u32,u32,u32
1,2021-03-16 00:00:00,93.0,-1.94,5.32,"""Nola, Aaron""",543807,605400,1,1,0,1,-12.348547,30.73969,-16.651987,2227.0,6.9,641572,53.58,1,1,0,0,0,0,0,0,231.0,0.0,null,5,0,0
5,2021-03-16 00:00:00,83.5,-2.0,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-14.455206,25.331797,-31.088718,1398.0,6.5,641572,53.98,1,2,0,0,0,0,0,0,256.0,0.0,null,1,1,0
1,2021-03-16 00:00:00,93.9,-1.89,5.3,"""Nola, Aaron""",543807,605400,1,2,0,1,-13.677609,29.965809,-15.211607,2175.0,7.0,641572,53.52,1,3,0,0,0,0,0,0,228.0,0.0,null,3,2,0
3,2021-03-16 00:00:00,79.6,-1.92,5.42,"""Nola, Aaron""",543807,605400,1,2,0,1,9.351337,24.548919,-39.815268,2603.0,6.6,641572,53.89,1,4,0,0,0,0,0,0,52.0,0.0,null,5,3,0
5,2021-03-16 00:00:00,83.8,-2.08,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-13.566679,23.380459,-33.125139,1382.0,6.6,641572,53.9,1,5,0,0,0,0,0,0,255.0,0.0,null,4,4,0


In [12]:
# release point consistency feature


df.head()

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,outs_when_up,inning,ax,ay,az,release_spin_rate,release_extension,game_pk,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,next_pitch,count,base_state
u32,datetime[μs],f64,f64,f64,str,i64,i64,u32,u32,i64,i64,f64,f64,f64,f64,f64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,u32,u32,u32
1,2021-03-16 00:00:00,93.0,-1.94,5.32,"""Nola, Aaron""",543807,605400,1,1,0,1,-12.348547,30.73969,-16.651987,2227.0,6.9,641572,53.58,1,1,0,0,0,0,0,0,231.0,0.0,null,5,0,0
5,2021-03-16 00:00:00,83.5,-2.0,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-14.455206,25.331797,-31.088718,1398.0,6.5,641572,53.98,1,2,0,0,0,0,0,0,256.0,0.0,null,1,1,0
1,2021-03-16 00:00:00,93.9,-1.89,5.3,"""Nola, Aaron""",543807,605400,1,2,0,1,-13.677609,29.965809,-15.211607,2175.0,7.0,641572,53.52,1,3,0,0,0,0,0,0,228.0,0.0,null,3,2,0
3,2021-03-16 00:00:00,79.6,-1.92,5.42,"""Nola, Aaron""",543807,605400,1,2,0,1,9.351337,24.548919,-39.815268,2603.0,6.6,641572,53.89,1,4,0,0,0,0,0,0,52.0,0.0,null,5,3,0
5,2021-03-16 00:00:00,83.8,-2.08,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-13.566679,23.380459,-33.125139,1382.0,6.6,641572,53.9,1,5,0,0,0,0,0,0,255.0,0.0,null,4,4,0


In [13]:
# # Late inning pressure (boolean)
# df = df.with_columns(
#     ((pl.col("inning") >= 7) & (pl.col("bat_score") - pl.col("fld_score")).abs() <= 3)
#     .cast(pl.Int32)
#     .alias("high_pressure")
# )

# df.head()

In [14]:
def sort_pitch_data(df):
    return df.sort(
        [
            "game_date",  # Primary: chronological order
            "game_pk",  # Secondary: unique game identifier
            "inning",  # Tertiary: game sequence
            "at_bat_number",  # Game at-bat order
            "pitch_number",  # At-bat pitch sequence
        ],
        descending=False,  # Ascending order for all
    )


df = sort_pitch_data(df)
df.head()

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,outs_when_up,inning,ax,ay,az,release_spin_rate,release_extension,game_pk,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,next_pitch,count,base_state
u32,datetime[μs],f64,f64,f64,str,i64,i64,u32,u32,i64,i64,f64,f64,f64,f64,f64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,u32,u32,u32
1,2021-03-16 00:00:00,93.0,-1.94,5.32,"""Nola, Aaron""",543807,605400,1,1,0,1,-12.348547,30.73969,-16.651987,2227.0,6.9,641572,53.58,1,1,0,0,0,0,0,0,231.0,0.0,null,5,0,0
5,2021-03-16 00:00:00,83.5,-2.0,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-14.455206,25.331797,-31.088718,1398.0,6.5,641572,53.98,1,2,0,0,0,0,0,0,256.0,0.0,null,1,1,0
1,2021-03-16 00:00:00,93.9,-1.89,5.3,"""Nola, Aaron""",543807,605400,1,2,0,1,-13.677609,29.965809,-15.211607,2175.0,7.0,641572,53.52,1,3,0,0,0,0,0,0,228.0,0.0,null,3,2,0
3,2021-03-16 00:00:00,79.6,-1.92,5.42,"""Nola, Aaron""",543807,605400,1,2,0,1,9.351337,24.548919,-39.815268,2603.0,6.6,641572,53.89,1,4,0,0,0,0,0,0,52.0,0.0,null,5,3,0
5,2021-03-16 00:00:00,83.8,-2.08,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-13.566679,23.380459,-33.125139,1382.0,6.6,641572,53.9,1,5,0,0,0,0,0,0,255.0,0.0,null,4,4,0


In [15]:
df.head()

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,outs_when_up,inning,ax,ay,az,release_spin_rate,release_extension,game_pk,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,next_pitch,count,base_state
u32,datetime[μs],f64,f64,f64,str,i64,i64,u32,u32,i64,i64,f64,f64,f64,f64,f64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,u32,u32,u32
1,2021-03-16 00:00:00,93.0,-1.94,5.32,"""Nola, Aaron""",543807,605400,1,1,0,1,-12.348547,30.73969,-16.651987,2227.0,6.9,641572,53.58,1,1,0,0,0,0,0,0,231.0,0.0,null,5,0,0
5,2021-03-16 00:00:00,83.5,-2.0,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-14.455206,25.331797,-31.088718,1398.0,6.5,641572,53.98,1,2,0,0,0,0,0,0,256.0,0.0,null,1,1,0
1,2021-03-16 00:00:00,93.9,-1.89,5.3,"""Nola, Aaron""",543807,605400,1,2,0,1,-13.677609,29.965809,-15.211607,2175.0,7.0,641572,53.52,1,3,0,0,0,0,0,0,228.0,0.0,null,3,2,0
3,2021-03-16 00:00:00,79.6,-1.92,5.42,"""Nola, Aaron""",543807,605400,1,2,0,1,9.351337,24.548919,-39.815268,2603.0,6.6,641572,53.89,1,4,0,0,0,0,0,0,52.0,0.0,null,5,3,0
5,2021-03-16 00:00:00,83.8,-2.08,5.26,"""Nola, Aaron""",543807,605400,1,1,0,1,-13.566679,23.380459,-33.125139,1382.0,6.6,641572,53.9,1,5,0,0,0,0,0,0,255.0,0.0,null,4,4,0


In [16]:
print(df.height)
filtered = []
for pitcher_df in df.group_by("pitcher"):
    pitcher_code = pitcher_df[0]
    pitcher_df = pitcher_df[1]
    val_counts = (
        pitcher_df.select(
            pl.col("pitch_type").value_counts(normalize=True).alias("pitch_type_counts")
        )
        .unnest("pitch_type_counts")
        .sort("proportion", descending=True)
    )

    # Get pitch types with proportion < 0.01
    low_prop_pitches = (
        val_counts.filter(pl.col("proportion") < 0.01)["pitch_type"].unique().to_list()
    )

    # Filter out those pitch types
    pitcher_df = pitcher_df.filter(~pl.col("pitch_type").is_in(low_prop_pitches))

    # combine back to main df
    filtered.append(pitcher_df)
# combine back to one df
df = pl.concat(filtered)

39819


In [19]:
batting_df = pb.batting_stats_bref(params["clean"]["start_year"])
player_ids = list(df.select("batter").unique().to_pandas()["batter"])
batting_df = pl.DataFrame(batting_df[batting_df["mlbID"].isin(player_ids)])
batting_df = batting_df.drop(
    [
        "Name",
        "Age",
        "#days",
        "Lev",
        "Tm",
        "G",
        "PA",
        "AB",
        "SO",
        "HBP",
        "SH",
        "SF",
        "SB",
        "CS",
    ]
)
df = df.join(batting_df, left_on="batter", right_on="mlbID", how="left")

df = df.fill_null(-1)
df = df.fill_nan(-1)

In [20]:
df.head(20)

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,type,outs_when_up,inning,ax,ay,az,release_spin_rate,release_extension,game_pk,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,next_pitch,count,base_state,R,H,2B,3B,HR,RBI,BB,IBB,GDP,BA,OBP,SLG,OPS
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64
1,2021-03-30 00:00:00,96.7,-1.44,6.41,"""Cease, Dylan""",606132,656302,0,1,0,1,-2.673928,31.850791,-8.134448,2600.0,6.1,644215,54.41,4,1,0,0,0,0,0,0,197.0,0.0,-1.0,3,0,0,69,133,26,2,6,50,40,2,8,0.273,0.327,0.372,0.699
3,2021-03-30 00:00:00,79.0,-1.44,6.17,"""Cease, Dylan""",606132,656302,0,1,0,1,3.914717,25.42844,-46.225336,2759.0,5.8,644215,54.67,4,2,0,0,0,0,0,0,21.0,0.0,-1.0,3,1,0,69,133,26,2,6,50,40,2,8,0.273,0.327,0.372,0.699
3,2021-03-30 00:00:00,79.4,-1.4,6.24,"""Cease, Dylan""",606132,656302,0,1,0,1,4.952816,24.615149,-46.935868,2814.0,6.0,644215,54.54,4,3,0,0,0,0,0,0,26.0,-0.025,-1.0,1,2,0,69,133,26,2,6,50,40,2,8,0.273,0.327,0.372,0.699
1,2021-03-30 00:00:00,97.6,-1.23,6.36,"""Cease, Dylan""",658069,656302,1,1,1,1,-11.212693,29.99058,-10.538991,2474.0,6.0,644215,54.51,5,1,0,0,0,0,0,0,199.0,0.0,-1.0,3,0,0,30,61,11,1,7,33,12,1,5,0.225,0.257,0.351,0.608
3,2021-03-30 00:00:00,78.6,-1.3,6.32,"""Cease, Dylan""",658069,656302,1,0,1,1,3.513302,22.855144,-45.664826,2681.0,5.9,644215,54.59,5,2,0,0,0,0,0,0,20.0,0.028,-1.0,1,1,0,30,61,11,1,7,33,12,1,5,0.225,0.257,0.351,0.608
1,2021-03-30 00:00:00,97.8,-1.69,6.27,"""Cease, Dylan""",596115,656302,1,1,1,1,-9.180623,32.523895,-10.515731,2595.0,6.2,644215,54.3,6,1,0,0,0,0,0,0,199.0,0.0,-1.0,2,0,2,88,132,34,5,24,75,53,2,7,0.251,0.329,0.472,0.801
2,2021-03-30 00:00:00,85.4,-2.08,6.09,"""Cease, Dylan""",596115,656302,1,1,1,1,6.946647,22.934122,-31.85811,3139.0,6.2,644215,54.32,6,2,0,0,0,0,0,0,69.0,0.0,-1.0,1,1,2,88,132,34,5,24,75,53,2,7,0.251,0.329,0.472,0.801
1,2021-03-30 00:00:00,97.9,-1.86,6.33,"""Cease, Dylan""",596115,656302,1,1,1,1,-7.237474,30.743734,-12.118229,2703.0,6.1,644215,54.4,6,3,0,0,0,0,0,0,196.0,0.0,-1.0,1,2,2,88,132,34,5,24,75,53,2,7,0.251,0.329,0.472,0.801
1,2021-03-30 00:00:00,98.6,-1.64,6.38,"""Cease, Dylan""",596115,656302,1,1,1,1,-3.925343,35.864846,-10.002543,2707.0,6.0,644215,54.46,6,4,0,0,0,0,0,0,192.0,-0.031,-1.0,3,2,2,88,132,34,5,24,75,53,2,7,0.251,0.329,0.472,0.801
